In [1]:
import tensorflow as tf
from tensorflow.keras.applications import resnet50, xception, inception_v3  
from keras import Input, Model, layers

In [2]:
import pathlib

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

data_dir = pathlib.Path('drive/My Drive/TIES4911/wonders_world/Wonders of World/Wonders of World')
(train_ds, val_ds) = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    shuffle=True,
    validation_split=0.2,
    subset='both',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=123,
    label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)

Found 3846 files belonging to 12 classes.
Using 3077 files for training.
Using 769 files for validation.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [4]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [5]:
inputs = Input(shape=(224, 224, 3))

base_model = inception_v3.InceptionV3(input_tensor=inputs, include_top=False)
base_model.trainable = False

preprocess_input = inception_v3.preprocess_input
global_average_layer = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(num_classes)

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = layers.Dropout(0.5)(x)
outputs = prediction_layer(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint = tf.train.Checkpoint(model=model)
model.summary()

loss0, accuracy0 = model.evaluate(val_ds)

87910968/87910968 [==============================] - 1s 0us/step


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [ ]:
print(len(model.trainable_variables))

2


In [ ]:
checkpoint_path = 'drive/My Drive/TIES4911/task4/task4-2/inceptionv3-{epoch:04d}'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=2*97,
    verbose=1)

history = model.fit(train_ds,
                    epochs=10,
                    validation_data=val_ds,
                    callbacks=[model_checkpoint_callback])

Epoch 1/10


97/97 [==============================] - 374s 4s/step - loss: 8.9200 - accuracy: 0.0715 - val_loss: 10.8823 - val_accuracy: 0.0923
Epoch 2/10
96/97 [============================>.] - ETA: 2s - loss: 9.6165 - accuracy: 0.0960
Epoch 2: saving model to drive/My Drive/TIES4911/task4/task4-2/inceptionv3-0002
97/97 [==============================] - 359s 4s/step - loss: 9.6218 - accuracy: 0.0962 - val_loss: 11.5909 - val_accuracy: 0.1443
Epoch 3/10
97/97 [==============================] - 344s 4s/step - loss: 9.6941 - accuracy: 0.1141 - val_loss: 10.3751 - val_accuracy: 0.1456
Epoch 4/10
96/97 [============================>.] - ETA: 2s - loss: 9.9935 - accuracy: 0.1195
Epoch 4: saving model to drive/My Drive/TIES4911/task4/task4-2/inceptionv3-0004
97/97 [==============================] - 358s 4s/step - loss: 10.0034 - accuracy: 0.1193 - val_loss: 10.5847 - val_accuracy: 0.1625
Epoch 5/10
97/97 [==============================] - 357s 4s/step - loss: 10.3007 - accuracy: 0.1212 - val_loss: 11.7

In [ ]:
loss1, accuracy1 = model.evaluate(val_ds)

25/25 [==============================] - 68s 3s/step - loss: 12.7016 - accuracy: 0.1443
